In [1]:
from utils import (
    extract_page_as_image, 
    parse_cells, 
    create_expanded_dataframe, 
    parse_command_line_arguments,
    prepare_counter,
    clean_text,
    create_parsed_dictionary,
)

import os
import pandas as pd
from bs4 import BeautifulSoup
from textractor import Textractor
from IPython.display import display, HTML, Image
from unstructured.partition.pdf import partition_pdf
from textractor.data.constants import TextractFeatures

In [2]:
extractor = Textractor(profile_name="default")

In [3]:
filename='test.pdf'

elements = partition_pdf(filename=filename, infer_table_structure=True, include_page_breaks=True, languages=['ita', 'eng'])
all_counter = prepare_counter(elements=elements)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
parsed_doc = {'page_1': ''}
curr_page = 'page_1'
page_number = 1
tables = []
n=0

for index, el in enumerate(elements):

    if el.category == 'PageBreak':
        page_number += 1
        curr_page = f"page_{page_number}"
        parsed_doc[curr_page] = ''

    elif el.category == 'ListItem':
        if all_counter[el.text] < 10 and all_counter[el.text] != 0:
            parsed_doc[curr_page] += el.text
            parsed_doc[curr_page] += '\n'
    
    elif el.category == 'NarrativeText':
        if all_counter[el.text] < 10 and all_counter[el.text] != 0:
            parsed_doc[curr_page] += el.text
            parsed_doc[curr_page] += ' '

    elif el.category == 'Table':
        parsed_doc[curr_page] += '\n'
        pdf_path = filename
        page_number = el.metadata.page_number 
        coords = el.metadata.coordinates.points 

        output_image_path = f'output_{n}.png'
        extract_page_as_image(pdf_path, page_number, output_image_path, coords)
        document = extractor.analyze_document(file_source=f'output_{n}.png',features=[TextractFeatures.TABLES])
        if len(document.tables) > 0:
            for k in range(len(document.tables)):
                raw_cells = document.tables[k].table_cells[:]  
                parsed_cells = parse_cells(raw_cells)
                df = create_expanded_dataframe(parsed_cells)
                df_cleaned = df.applymap(clean_text)
                df_cleaned.columns = [clean_text(col) for col in df_cleaned.columns]
                tables.append(df_cleaned)
                parsed_doc[curr_page] += df_cleaned.to_markdown(index=False)
                parsed_doc[curr_page] += '\n'
                n+=1
        else:
            pass
        

    elif el.category == 'Text':
        if all_counter[el.text] < 10 and all_counter[el.text] != 0:
            parsed_doc[curr_page] += el.text
            parsed_doc[curr_page] += ' '

    elif el.category == 'Title':

        if page_number == 1:
            parsed_doc[curr_page] += el.text
            parsed_doc[curr_page] += '\n'
        elif all_counter[el.text] < 10 and all_counter[el.text] != 0:
            parsed_doc[curr_page] += el.text
            parsed_doc[curr_page] += '\n'

parsed_doc = {k:v for k,v in parsed_doc.items() if v} 

In [5]:
im = [archivo for archivo in os.listdir('.') if os.path.isfile(archivo) and archivo.lower().endswith('.png')]
images = sorted(im)
len(images)

8

In [8]:


html_output = ""
for image_path, df in zip(images, tables):
    table_html = df.to_html(index=False)
    
    image_html = f'<img src="{image_path}" style="display: inline-block; vertical-align: top; margin-right: 20px; width: 300px;">'
    table_html = f'<div style="display: inline-block; vertical-align: top; width: 60%;">{table_html}</div>'
    
    html_output += f'<div style="margin-bottom: 20px; width: 100%; display: flex;">{image_html}{table_html}</div>'

display(HTML(html_output))
